In [4]:
import os
import copy
from dataclasses import dataclass
import numpy as np
import torch
from datasets import Dataset, load_dataset, concatenate_datasets
from datasets.features import Value
from transformers import (
    BitsAndBytesConfig,
    AutoModelForSequenceClassification, 
    AutoTokenizer,  
    TrainingArguments,
    DataCollatorWithPadding,
    Trainer,
    EvalPrediction,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score
from transformers.integrations import TensorBoardCallback
from tqdm import tqdm

In [5]:
# parameters here for training and lora config 
class Config:
    output_dir: str = "llama-3.2-3B-Instruct"
    checkpoint: str = "meta-llama/Llama-3.2-3B" 
    max_length: int = 2048
    n_splits: int = 5
    fold_idx: int = 0
    optim_type: str = "adamw_8bit"
    per_device_train_batch_size: int = 2 # reduce 1 
    gradient_accumulation_steps: int = 2  
    per_device_eval_batch_size: int = 8
    n_epochs: int = 3
    freeze_layers: int = 0 # changed to 0 
    lr: float = 2e-4 #  LR=2e-4 1 epoch linear schedule with warmup, then a=4 is best alpha for all rank. 
    warmup_steps: int = 20
    lora_r: int = 64  # changed to 64, or 1024
    lora_alpha: float = 4 # changed to 16, or 4 next 
    lora_dropout: float = 0.05
    lora_bias: str = "none"
    
config = Config()

In [6]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    # only target self-attention
    target_modules=["q_proj", "k_proj", "v_proj", "down_proj","up_proj","o_proj","gate_proj"], # added more target layers
    layers_to_transform=[i for i in range(42) if i >= config.freeze_layers],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS,
)

In [7]:
# instantiate tokenizer and model 
tokenizer = AutoTokenizer.from_pretrained(config.checkpoint)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.padding_side = "right" 

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=3,
    torch_dtype=torch.bfloat16, 
    device_map="auto",
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.config.pad_token_id = model.config.eos_token_id
model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-3B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
   

In [9]:
model.print_trainable_parameters()

trainable params: 97,264,640 || all params: 3,310,023,680 || trainable%: 2.9385


In [10]:
dataset = load_dataset("lmsys/lmsys-arena-human-preference-55k")
current_features = dataset['train'].features
current_features['id'] = Value('string')
dataset['train'] = dataset['train'].cast(current_features)
dataset['train'].features

{'id': Value(dtype='string', id=None),
 'model_a': Value(dtype='string', id=None),
 'model_b': Value(dtype='string', id=None),
 'prompt': Value(dtype='string', id=None),
 'response_a': Value(dtype='string', id=None),
 'response_b': Value(dtype='string', id=None),
 'winner_model_a': Value(dtype='int64', id=None),
 'winner_model_b': Value(dtype='int64', id=None),
 'winner_tie': Value(dtype='int64', id=None)}

In [11]:
dataset_33k = Dataset.from_csv("lmsys-33k-deduplicated.csv")
dataset_33k.features

{'id': Value(dtype='string', id=None),
 'model_a': Value(dtype='string', id=None),
 'model_b': Value(dtype='string', id=None),
 'prompt': Value(dtype='string', id=None),
 'response_a': Value(dtype='string', id=None),
 'response_b': Value(dtype='string', id=None),
 'winner_model_a': Value(dtype='int64', id=None),
 'winner_model_b': Value(dtype='int64', id=None),
 'winner_tie': Value(dtype='int64', id=None)}

In [12]:
dataset = concatenate_datasets([dataset['train'], dataset_33k])
dataset

Dataset({
    features: ['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b', 'winner_model_a', 'winner_model_b', 'winner_tie'],
    num_rows: 78664
})

In [13]:
unique_model_ids = set(dataset['model_a']).union(set(dataset['model_b']))
unique_model_ids_list = list(unique_model_ids)
print(unique_model_ids_list[0], unique_model_ids_list[1])
print(type(unique_model_ids_list[0]))

gpt-3.5-turbo codellama-34b-instruct
<class 'str'>


In [14]:
class CustomTokenizer:
    def __init__(
        self, 
        tokenizer: tokenizer, 
        model, 
        max_length: int,
        unique_model_ids_list: list 
    ) -> None:
        self.tokenizer = tokenizer
        self.model = model 
        self.max_length = max_length

        special_tokens_dict = {"additional_special_tokens": unique_model_ids_list}
        self.tokenizer.add_special_tokens(special_tokens_dict)
        self.model.resize_token_embeddings(len(self.tokenizer))
        
    def __call__(self, batch: dict) -> dict:
        prompt = ["<prompt>: " + self.process_text(t) for t in batch["prompt"]]

        response_a = [f"\n\nresponse of {t}: " + self.process_text(r) for t, r in zip(batch["model_a"], batch["response_a"])]
        response_b = [f"\n\nresponse of {t}: " + self.process_text(r) for t, r in zip(batch["model_b"], batch["response_b"])]

        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)
        
        labels = []
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):
            if a_win:
                label = 0
            elif b_win:
                label = 1
            else:
                label = 2
            labels.append(label)
        
        return {**tokenized, "labels": labels}

    @staticmethod
    def process_text(text: str) -> str:
        return " ".join(eval(text, {"null": ""}))

In [12]:
import torch

embedding_layer = model.get_input_embeddings()
special_token_ids = [tokenizer.convert_tokens_to_ids(token) for token in unique_model_ids_list]

embedding_layer.weight.requires_grad = True

for index, token_id in enumerate(special_token_ids):
    embedding = embedding_layer.weight[token_id]
    print(f"Index {index+1} - Token ID {token_id}:")
    print(f"Embedding values: {embedding}")
    print(f"Requires grad: {embedding_layer.weight.requires_grad}\n")

Index 1 - Token ID None:
Embedding values: tensor([[[ 0.0045,  0.0166,  0.0210,  ..., -0.0054, -0.0422, -0.0315],
         [ 0.0215, -0.0238,  0.0211,  ..., -0.0107, -0.0011, -0.0374],
         [ 0.0136,  0.0104,  0.0128,  ...,  0.0081, -0.0122,  0.0051],
         ...,
         [ 0.0009,  0.0164, -0.0193,  ..., -0.0003, -0.0030,  0.0066],
         [ 0.0009,  0.0164, -0.0193,  ..., -0.0003, -0.0030,  0.0066],
         [ 0.0009,  0.0164, -0.0193,  ..., -0.0003, -0.0030,  0.0066]]],
       device='cuda:0', grad_fn=<UnsqueezeBackward0>)
Requires grad: True

Index 2 - Token ID None:
Embedding values: tensor([[[ 0.0045,  0.0166,  0.0210,  ..., -0.0054, -0.0422, -0.0315],
         [ 0.0215, -0.0238,  0.0211,  ..., -0.0107, -0.0011, -0.0374],
         [ 0.0136,  0.0104,  0.0128,  ...,  0.0081, -0.0122,  0.0051],
         ...,
         [ 0.0009,  0.0164, -0.0193,  ..., -0.0003, -0.0030,  0.0066],
         [ 0.0009,  0.0164, -0.0193,  ..., -0.0003, -0.0030,  0.0066],
         [ 0.0009,  0.0164, 

In [ ]:
encode = CustomTokenizer(tokenizer, model, max_length=config.max_length, unique_model_ids_list=unique_model_ids_list)
dataset = dataset.map(encode, batched=True)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Map:   0%|          | 0/78664 [00:00<?, ? examples/s]

<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequence '\/'
<string>:1: SyntaxWarning: invalid escape sequen

In [ ]:
print(dataset)
sample = dataset[0]

decoded_input = tokenizer.decode(sample['input_ids'], skip_special_tokens=False)
print("Decoded Input: ", decoded_input)

In [ ]:
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions
    labels = eval_preds.label_ids
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()
    loss = log_loss(y_true=labels, y_pred=probs)
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))
    return {"acc": acc, "log_loss": loss}

In [ ]:
eval_idx = [i for i in range(57477) if i % 5 == 0]  # Get every 5th index
train_idx = [i for i in range(len(dataset)) if i not in eval_idx]

print(f"Number of evaluation indices: {len(eval_idx)}")
print(f"Number of training indices: {len(train_idx)}")

In [ ]:
eval_dataset=dataset.select(eval_idx)
eval_dataframe = eval_dataset.to_pandas()
eval_dataframe.head()

In [ ]:
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    report_to="tensorboard",  
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=10,  # Log every 10 steps
    logging_dir='./logs/llama-3.2-instruct-model-ids', 
    eval_strategy="steps",  
    eval_steps=5000,  
    save_strategy="steps",
    save_steps=5000,
    metric_for_best_model="eval_acc", 
    optim=config.optim_type,
    fp16=True,
    learning_rate=config.lr,
    warmup_steps=config.warmup_steps,
)

In [ ]:
trainer = Trainer(
    args=training_args, 
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset.select(train_idx),
    eval_dataset=dataset.select(eval_idx),
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    callbacks=[TensorBoardCallback()],
)
trainer.train()

In [ ]:
eval_dataset=dataset.select(eval_idx)
predictions = trainer.predict(eval_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)  # Get the predicted class (3-way classification)
true_labels = predictions.label_ids  # Get the true class labels

# Compute confusion matrix
cm = confusion_matrix(true_labels, pred_labels, labels=[0, 1, 2])

# Plotting the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Model A Win', 'Model B Win', 'Tie'])
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix for 3-Way Model Classification')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()